# TF-IDF with Scikit Learn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
import nltk
import glob
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore')

C:\Users\Daniel\anaconda3\envs\lyrics_analysis\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
# load all the dataframes
country_df = pd.read_csv('data/Country-lyrics.csv', index_col=0)
hip_hop_df = pd.read_csv('data/Hip-Hop-Rnb-lyrics.csv', index_col=0)
pop_df = pd.read_csv('data/pop-lyrics.csv', index_col=0)
christian_df = pd.read_csv('data/Christian-lyrics.csv', index_col=0)
electro_df = pd.read_csv('data/Electro-lyrics.csv', index_col=0)
rock_df = pd.read_csv('data/Rock-lyrics.csv', index_col=0)

In [3]:
# empyty list for documents
documents = []

In [4]:
# create documents of country music song lyrics
for key, values in country_df.groupby(['artist', 'title', 'year']):
    documents.append(values['word'].astype('str').values)

documents = [' '.join(doc) for doc in documents]

In [5]:
# remove song title and 'lyrics' from start of each document
# remove 'embed' that is at the end of each document
# remove punctuation but keep apostrophes
documents = [re.sub("\d{1,4}Embed|Embed|^.*?(Lyrics)|[^\w\d'\s]+" , '', doc) for doc in documents]

<>:4: DeprecationWarning: invalid escape sequence \d
<>:4: DeprecationWarning: invalid escape sequence \d
C:\Users\Daniel\AppData\Local\Temp\ipykernel_20452\3280666032.py:4: DeprecationWarning: invalid escape sequence \d
  documents = [re.sub("\d{1,4}Embed|Embed|^.*?(Lyrics)|[^\w\d'\s]+" , '', doc) for doc in documents]


In [6]:
def lemmatization(documents, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in documents:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [7]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    max_features=100,
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 5),
    stop_words='english'
)

In [8]:
lemmatized_documents = lemmatization(documents)

vectors = vectorizer.fit_transform(lemmatized_documents)

feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

In [9]:
for description in denselist:
    i = 0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[i])
        i += 1
    all_keywords.append(keywords)

In [10]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(all_keywords)

texts = all_keywords

corpus = [id2word.doc2bow(text) for text in texts]

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.055291  0.050191       1        1  54.501694
0     -0.026004 -0.066461       2        1  45.471529
2     -0.029287  0.016271       3        1   0.026776, topic_info=               Term        Freq       Total Category  logprob  loglift
44              way  365.000000  365.000000  Default  30.0000  30.0000
69              day  241.000000  241.000000  Default  29.0000  29.0000
28         low like  490.000000  490.000000  Default  28.0000  28.0000
39  ticket low like  490.000000  490.000000  Default  27.0000  27.0000
74             look  309.000000  309.000000  Default  26.0000  26.0000
..              ...         ...         ...      ...      ...      ...
94            drive    0.060255  162.258318   Topic3  -4.6052   0.3270
95             away    0.060255  181.597011   Topic3  -4.6052   0.2144
2              come    0.060255  399.213783   Topic3  -4.6052  -0.5733
3             drink    0.060255  168.180289   Topic3  -4.6052   0.2912
4              good    0.060255  365.004196   Topic3  -4.6052  -0.4837

[173 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
95        1  0.958166   away
95        2  0.044054   away
85        1  0.454702   baby
85        2  0.543916   baby
66        1  0.640336    bar
...     ...       ...    ...
64        2  0.125214   wish
65        1  0.131650  world
65        2  0.870915  world
76        1  0.910211  wrong
76        2  0.089042  wrong

[182 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])